# Publish data

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import netCDF4 as nc

import meop
import meop_publish
import meop_filenames
import meop_metadata
import meop_plot_data
from importlib import reload


In [85]:
# extract WMO numbers from platform3
df = pd.read_json('~/disk2_roquet/RAW_MEOP_DATA/config_files/platform2.json')
df['wmo_platform_code'].where(~df.wmo_platform_code.isnull(),0,inplace=True)
df['wmo'] = df.apply(lambda x: f"Q99{int(x['wmo_platform_code']):05d}", axis=1)
df['wmo'].replace('Q9900000',' ',inplace=True)

df2 = pd.read_json('~/disk2_roquet/RAW_MEOP_DATA/config_files/platform2_patch.json')
df2['wmo'] = df2['wmo_platform_code']
df2['wmo'].fillna(' ',inplace=True)

df = pd.concat([df,df2])
df = df.drop(columns=['wmo_platform_code','species','location','firmware_version','firmware_parameters','loc_algorithm','dt_created','dt_modified'])
df

,platform_code,smru_platform_code,deployment_code,time_coverage_start,time_coverage_end,instr_id,ptt,wmo
0,10320,wd1-9497-06,wd1,2006-02-14T00:00:00Z,2006-11-01T00:00:00Z,9497,19251,
1,10332,bs2-53130-05,bs2,2006-10-19T00:00:00Z,2007-05-01T00:00:00Z,10286,53130,
2,10338,ct10r-24176-05,ct10r,2005-02-01T00:00:00Z,2005-06-01T00:00:00Z,10059,924176,
3,10344,ct10r-24178-05,ct10r,2005-02-01T00:00:00Z,2005-06-01T00:00:00Z,10057,924178,
4,10350,ct10r-25966-05,ct10r,2005-02-01T00:00:00Z,2005-07-01T00:00:00Z,10061,925966,
...,...,...,...,...,...,...,...,...
314,183709,ct159-943-BAT-19,ct159,2019-10-24T00:00:00Z,None,14943,183709,Q9901404
315,67077,ct159-969-BAT-18,ct159,2019-10-24T00:00:00Z,None,14969,67077,Q9901425
316,183711,ct159-051-BAT-19,ct159,2019-10-24T00:00:00Z,None,15051,183711,Q9901405
317,183707,ct159-052-BAT-19,ct159,2019-10-24T00:00:00Z,None,15052,183707,Q9901403


In [86]:
# select only rows present in the public MEOP-CTD database
publicdir_CTD = meop_filenames.publicdir /meop_filenames.version
lprofiles, ltags, ldeployments = meop_publish.load_list_profiles(publicdir_CTD, public=True, rebuild=False)
ltags.SMRU_PLATFORM_CODE

0         awru1-A-06
1         awru1-B-06
2         awru1-C-06
3       bs1-53127-04
4       bs1-53128-04
            ...     
1370     wd10-678-17
1371     wd10-679-17
1372     wd10-681-17
1373     wd3-CTD2-07
1374     wd3-CTD3-07
Name: SMRU_PLATFORM_CODE, Length: 1375, dtype: object

In [87]:
df[df.smru_platform_code.isin(ltags.SMRU_PLATFORM_CODE)].reset_index(drop=True).to_csv('../list_wmo_numbers.csv',index=False)